In [1]:
import time
import cv2
import numpy as np
import torchvision
import PIL
from fastai.vision import *

In [2]:
path = r'C:\Users\onlyt\webcam\emojii\smileface.png'
img = cv2.imread(path,-1)
img_counter = 0

In [3]:
def face_detection(img):
    FACE_CLASSIFIER = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    EYE_CLASSIFIER = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    faces = FACE_CLASSIFIER.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h+10),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
#         eyes = EYE_CLASSIFIER.detectMultiScale(roi_gray)
#         for(ex,ey,ew,eh) in eyes:
#             cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    return img[y:y+h, x:x+w]

In [4]:
#emojii
emojii = []
arrows0 = cv2.imread("emojii/smileface.png")
arrows = cv2.resize(arrows0,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_CUBIC)
arrows = cv2.resize(arrows0,(100,100),interpolation=cv2.INTER_AREA)
emojii.append(arrows)

arrows0 = cv2.imread("emojii/sadface.png")
arrows = cv2.resize(arrows0,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_CUBIC)
arrows = cv2.resize(arrows0,(100,100),interpolation=cv2.INTER_AREA)
emojii.append(arrows)

arrows0 = cv2.imread("emojii/normalface.jpg")
arrows = cv2.resize(arrows0,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_CUBIC)
arrows = cv2.resize(arrows0,(100,100),interpolation=cv2.INTER_AREA)
emojii.append(arrows)

In [ ]:
learn = load_learner('dataset/models')
cv2.namedWindow("preview")

vc = cv2.VideoCapture(0)

x_offset=y_offset=10
c = 0
img_counter = 0
choosen_emojii = None

if vc.isOpened(): # try to get the first frame
    rval, frame = vc.read()
else:
    rval = False

while rval:
#     cv2.imshow("preview", frame)
    rval, frame = vc.read()
    try:
        img = face_detection(frame)
        c = c + 1
        if c == 10:
            img = PIL.Image.fromarray(img).convert('LA')
            img = Image(pil2tensor(img.convert("RGB"), np.float32).div_(255))
            pred_class,pred_idx,outputs = learn.predict(img)
            pred_class = str(pred_class)

            if pred_class == 'smile':
                choosen_emojii = 0
            elif pred_class=='sad':
                choosen_emojii = 1
            elif pred_class == 'normal':
                choosen_emojii = 2
            c=0
    except:
        pass
        
    if choosen_emojii != None:
        frame[y_offset:y_offset+emojii[2].shape[0], x_offset:x_offset+emojii[2].shape[1]]=emojii[choosen_emojii]    
    cv2.imshow("preview", frame)
    key = cv2.waitKey(20)
    if key == 27: # exit on ESC
        out = img
        break
    elif key%256 == 32:
        # SPACE pressed
        img_name = "dataset/opencv_frame_{}.png".format(img_counter)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(img_name, img)
        print("{} written!".format(img_name))
        img_counter += 1
cv2.destroyWindow("preview")